# Практическое задание

## Урок 4. Система управления базами данных MongoDB в Python


### 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии в вашу базу.

In [1]:
# импортируем необходимые библиотеки и модули

from pprint import pprint
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from time import sleep

# импортируем pymongo для работы с MongoDB
from pymongo import MongoClient

In [2]:
# название сессии
client = MongoClient()

# создадим базу данных для вакансий
vacancy_db = client.vacancy_hh_ru_db
vacancy_db

# создаем коллекцию для текущего сайта 
vacancy_hh_ru = vacancy_db.hh_ru 
vacancy_hh_ru

# курсор для коллекции vacancy_hh_ru
vacancy_hh_ru_cursor = vacancy_hh_ru.find()

In [3]:
# проверим какие базы данных имеются
client.list_database_names()

['admin', 'config', 'local']

In [4]:
# vacancy_db не выводится, так как пока не содержит данных 

In [5]:
# создадим списки для параметров вакансий

# наименовние вакансии  
job_vacancy_list = []

# зарплата 
salary_list = []

# минимальная зарплата
min_salary_list = []

# максимальная зарплата 
max_salary_list = []

# валюта зарплаты
currency_salary_list = []

# работодатель 
employers_list = []

# месторасположение
location_list = []

# ссылка на вакансию
link_title_list = []

# сайт вакансии
site_job_title_list = []

In [6]:
# создаем переменную для начальной страницы поиска
num_page = 0

# создаем переменную, содержащую общее количество найденных страниц
num_site_pages = 1

In [7]:
# функция, которая проверяет вакансии на дубли и добавляет только новые

def repeat_check_vacancy (df):
    # создадим цикл для добавления каждого элемента словаря в коллекцию 'vacancy_hh_ru' через 'insert_one'
    for vacancy in dict_data_job_hh_ru_df:
        try:
            select_vacancy = vacancy_hh_ru.find(
                {
                    '$and': [{'наименовние вакансии': {'$eq': vacancy['наименовние вакансии']}},
                             {'минимальная зарплата': {'$eq': vacancy['минимальная зарплата']}},
                             {'максимальная зарплата': {'$eq': vacancy['максимальная зарплата']}},
    #                          {'валюта зарплаты': {'$eq': vacancy['валюта зарплаты'}},
    #                          {'ссылка на вакансию': {'$eq': vacancy['ссылка на вакансию'}},
                             {'работодатель': {'$eq': vacancy['работодатель']}},
                             {'месторасположение': {'$eq': vacancy['месторасположение']}}]           

                }
            )

            i = 0

            for vac in select_vacancy:           
                i += 1
            # если вакансия не дублируется, то она добавляется в базу данных
            if i == 0:
                vacancy_hh_ru.insert_one(vacancy)
            else:
                # сообщение об ошибке и вывод информации о вакансии в случае дублирования
                print(f'ошибка! вакансия - {vacancy} дублируется, не добавлена')
        except Exceprion as e:
            print(e)
            
# цикл для каждой страницы поиска 
for page in range(num_site_pages):
    URL_HH_RU = f'https://hh.ru/search/vacancy?text=python&area=3&page={num_page}' 
    USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

    headers = {
    'User-Agent' : USER_AGENT
    }

    response = requests.get(url=URL_HH_RU, headers=headers)
    dom = bs(response.text, 'html.parser')

    job_vacancy = dom.find('div', {'id': 'a11y-main-content'}) # div с карточками вакансий - родительский 

    # наименовние вакансии 
    for job in job_vacancy:
        job = job.find('a', {'class': 'serp-item__title'})
        if job is not None:
            job_vacancy_list.append(job.text)
        else:
            job_vacancy_list.append(job)

    # зарплата 
    salary = job_vacancy.find('span', {'class': 'bloko-header-section-3'}) 
    for el_sal in job_vacancy:
        el_sal = el_sal.find('span', {'class': 'bloko-header-section-3'})
        if el_sal is None:
            el_sal = 'не указана'
            salary_list.append(el_sal)
        else:
            el_sal = el_sal.text.strip()
            el_sal = el_sal.replace('руб.', '')
            el_sal = el_sal.replace('\u202f', '')
            el_sal = el_sal.replace('от', '')
            el_sal = el_sal.replace('до ', '')
            salary_list.append(el_sal)


    # создаем две копии salary_list для разделения на минимальную и максимальную зарплату
    salary_list_min_prep = salary_list.copy()
    salary_list_max_prep = salary_list.copy()


    # минимальная зарплата
    for index, el in enumerate(salary_list_min_prep, 0):
        if type(el) is not int:
            salary_list_min_prep[index] = el.split(' – ')[0]
            try:
                salary_list_min_prep[index] = int(salary_list_min_prep[index])
            except:
                salary_list_min_prep[index] = salary_list_min_prep[index]

        min_salary_list.append(salary_list_min_prep[index])
    salary_list_min_prep = []


    # максимальная зарплата 
    for index, el in enumerate(salary_list_max_prep, 0):
        if type(el) is not int:
            salary_list_max_prep[index] = el.split(' – ')[-1]
            try:
                salary_list_max_prep[index] = int(salary_list_max_prep[index])
            except:
                salary_list_max_prep[index] = salary_list_max_prep[index]

        max_salary_list.append(salary_list_max_prep[index])
    salary_list_max_prep = []
    salary_list = []

    # работодатель
    for employer in job_vacancy:
        employer = employer.find('a', {'class': 'bloko-link bloko-link_kind-tertiary'})
        if employer is None:
            employer = 'None'
            employers_list.append(employer)
        else:
            employer = employer.text
            employer = employer.strip()
            employer = employer.replace('\xa0', ' ')
            employers_list.append(employer)

    # месторасположение
    for location in job_vacancy:
        location = location.find('div', {'data-qa': 'vacancy-serp__vacancy-address'})
        if location is None:
            emplolocationyer = 'None'
            location_list.append(location)
        else:
            location = location.text
            location = location.strip()
            location = location.replace('\xa0', ' ')
            location_list.append(location)            
            
            
    # ссылка на вакансию
    for link in job_vacancy:
        link = link.find('a', {'class' : 'serp-item__title'})
        if link is None:
            link = "None"
            link_title_list.append(link)
        else:
            link = link['href']
            link_title_list.append(link)
            
    # напечатаем номер страницы, код состояния и ссылку
    pprint(f'page {num_page} - ok, {response.status_code}, {response.url}')
    
    # увеличим номер страницы на 1 для перехода к следующей
    num_page += 1
    
    sleep(0.5)
            
 # валюта зарплаты и сайт вакансии 

for el in job_vacancy_list:
    cur = 'руб.'
    currency_salary_list.append(cur)
    site = 'https://ekaterinburg.hh.ru/'
    site_job_title_list.append(site)

    
dict_data_job_hh_ru = {
    'наименовние вакансии' : job_vacancy_list,
    'минимальная зарплата' : min_salary_list,
    'максимальная зарплата': max_salary_list,
    'валюта зарплаты': currency_salary_list,
    'ссылка на вакансию': link_title_list,
    'работодатель': employers_list,
    'месторасположение' : location_list           
}

dict_data_job_hh_ru_df = pd.DataFrame(dict_data_job_hh_ru)
dict_data_job_hh_ru_df = dict_data_job_hh_ru_df.dropna().reset_index(drop=True)
dict_data_job_hh_ru_df['валюта зарплаты'] = np.where((dict_data_job_hh_ru_df['минимальная зарплата'] == 'не указана'),
                                                     'не указана', dict_data_job_hh_ru_df['валюта зарплаты'])

# преобразуем датафрейм в словарь при помощи метода 'records'

dict_data_job_hh_ru_df = dict_data_job_hh_ru_df.to_dict('records')
repeat_check_vacancy (df=dict_data_job_hh_ru_df)

('page 0 - ok, 200, '
 'https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=0')


In [8]:
# проверим еще раз какие базы данных имеются
client.list_database_names()

['admin', 'config', 'local', 'vacancy_hh_ru_db']

In [9]:
# проверим коллекции
vacancy_db.list_collection_names()

['hh_ru']

In [10]:
# содержимое коллекции
for vac in vacancy_hh_ru.find():
    pprint(vac)

{'_id': ObjectId('63f26c18d290a18c2a48e5b2'),
 'валюта зарплаты': 'руб.',
 'максимальная зарплата': 39000,
 'месторасположение': 'Екатеринбург',
 'минимальная зарплата': 15000,
 'наименовние вакансии': 'Специалист службы поддержки с техническими знаниями '
                         '(TOOLS)',
 'работодатель': 'Яндекс',
 'ссылка на вакансию': 'https://ekaterinburg.hh.ru/vacancy/73595106?from=vacancy_search_list&query=python'}
{'_id': ObjectId('63f26c18d290a18c2a48e5b3'),
 'валюта зарплаты': 'не указана',
 'максимальная зарплата': 'не указана',
 'месторасположение': 'Екатеринбург',
 'минимальная зарплата': 'не указана',
 'наименовние вакансии': 'Программист Python Junior',
 'работодатель': 'BRANDPOL',
 'ссылка на вакансию': 'https://ekaterinburg.hh.ru/vacancy/77146976?from=vacancy_search_list&query=python'}
{'_id': ObjectId('63f26c18d290a18c2a48e5b4'),
 'валюта зарплаты': 'руб.',
 'максимальная зарплата': 340000,
 'месторасположение': 'Екатеринбург',
 'минимальная зарплата': 220000,
 'наи

In [11]:
# проверим работу функции (которая реализована выше) - попытаемся добавить вакансию, которая уже есть в базе данных 

def repeat_check_vacancy (df):
    # создадим цикл для добавления каждого элемента словаря в коллекцию 'vacancy_hh_ru' через 'insert_one'
    for vacancy in df:
        try:
            select_vacancy = vacancy_hh_ru.find(
                {
                    '$and': [{'наименовние вакансии': {'$eq': vacancy['наименовние вакансии']}},
                             {'минимальная зарплата': {'$eq': vacancy['минимальная зарплата']}},
                             {'максимальная зарплата': {'$eq': vacancy['максимальная зарплата']}},
    #                          {'валюта зарплаты': {'$eq': vacancy['валюта зарплаты'}},
    #                          {'ссылка на вакансию': {'$eq': vacancy['ссылка на вакансию'}},
                             {'работодатель': {'$eq': vacancy['работодатель']}},
                             {'месторасположение': {'$eq': vacancy['месторасположение']}}]           

                }
            )

            i = 0

            for vac in select_vacancy:           
                i += 1
            # если вакансия не дублируется, то она добавляется в базу данных
            if i == 0:
                vacancy_hh_ru.insert_one(vacancy)
            else:
                # сообщение об ошибке и вывод информации о вакансии в случае дублирования
                pprint(f'ошибка! вакансия - {vacancy} дублируется, не добавлена')
        except Exceprion as e:
            print(e)
            
vacancy_test_cheack_repeat = [{
 'валюта зарплаты': 'руб.',
 'максимальная зарплата': 39000,
 'месторасположение': 'Екатеринбург',
 'минимальная зарплата': 15000,
 'наименовние вакансии': 'Специалист службы поддержки с техническими знаниями '
                         '(TOOLS)',
 'работодатель': 'Яндекс',
 'ссылка на вакансию': 'https://ekaterinburg.hh.ru/vacancy/73595106?from=vacancy_search_list&query=python'}]
df_test_cheack_repeat = pd.DataFrame(vacancy_test_cheack_repeat)
df_test_cheack_repeat = df_test_cheack_repeat.to_dict('records')


repeat_check_vacancy (df=df_test_cheack_repeat)

("ошибка! вакансия - {'валюта зарплаты': 'руб.', 'максимальная зарплата': "
 "39000, 'месторасположение': 'Екатеринбург', 'минимальная зарплата': 15000, "
 "'наименовние вакансии': 'Специалист службы поддержки с техническими знаниями "
 "(TOOLS)', 'работодатель': 'Яндекс', 'ссылка на вакансию': "
 "'https://ekaterinburg.hh.ru/vacancy/73595106?from=vacancy_search_list&query=python'} "
 'дублируется, не добавлена')


### 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [12]:
# функция для поиска и вывода вакансии с заработной платой больше введенной суммы по полям "максимальная зарплата"
# и "минимальная зарлпата"

def salary_vacancy(input_salary):
    i = 0 
    select_salary_vacancy = vacancy_hh_ru.find({'$or': [{'минимальная зарплата': {'$gt': input_salary}}, 
                                                        {'максимальная зарплата': {'$gt': input_salary}}]})
    for vac in select_salary_vacancy:
        pprint(vac)
        i += 1
    if i == 0:
        pprint(f'вакансии с максимальной или минимальной зарплатой - {input_salary} не найдены')

In [13]:
salary_vacancy(input_salary=50000)

{'_id': ObjectId('63f26c18d290a18c2a48e5b4'),
 'валюта зарплаты': 'руб.',
 'максимальная зарплата': 340000,
 'месторасположение': 'Екатеринбург',
 'минимальная зарплата': 220000,
 'наименовние вакансии': 'Ведущий Python-developer',
 'работодатель': 'Тензор',
 'ссылка на вакансию': 'https://ekaterinburg.hh.ru/vacancy/77013935?from=vacancy_search_list&query=python'}
{'_id': ObjectId('63f26c18d290a18c2a48e5b5'),
 'валюта зарплаты': 'руб.',
 'максимальная зарплата': 200000,
 'месторасположение': 'Екатеринбург',
 'минимальная зарплата': 100000,
 'наименовние вакансии': 'Python разработчик',
 'работодатель': 'Соловьев Игорь',
 'ссылка на вакансию': 'https://ekaterinburg.hh.ru/vacancy/76786188?from=vacancy_search_list&query=python'}
{'_id': ObjectId('63f26c18d290a18c2a48e5b7'),
 'валюта зарплаты': 'руб.',
 'максимальная зарплата': 250000,
 'месторасположение': 'Екатеринбург, Геологическая и еще 1',
 'минимальная зарплата': 200000,
 'наименовние вакансии': 'Team leader (Python)',
 'работодател

In [14]:
salary_vacancy(input_salary=500000)

'вакансии с максимальной или минимальной зарплатой - 500000 не найдены'
